## 1. Import the main libraries

In [2]:
# import main ML libraries
# Python ≥3.5
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20
import sklearn
assert sklearn.__version__ >= "0.20"

import os
import pandas as pd
import numpy as np

# matplotlib import
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns

# Save figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

# from google.colab import drive
# drive.mount('/content/drive')

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## 2. Import the dataset

In [4]:
# import the dataset
df = pd.read_csv('data_dropouts.csv', delimiter = ";")

In [5]:
# Check the total number of columns, missing values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

We've got **4424** rows and **37** columns, with any missing value.

We've got only *int* and *float* data types among features and an object data type as our target. This is a major problem to address, because many of these features are encoded categorical features, many of them with multiple levels. The only continuous features are:
- Previous qualification (grade)
- Admission grade
- Unemployment rate
- Inflation rate
- GDP

So we have the 86% of features that are categorical.

Moreover, the **target** isn't binary, and this is another problem we need to address. We may choose to drop one of them, if it doesn't relate to sufficient entrances; or we can merge two of them, if we don't loose relevant informations about the phenomenon we're analysing.

Finally, we need to manipulate columns' names in order to work faster with them and correct the name of the "**Nacionality**" feature.

In [7]:
# Rename the column Nationality
df.rename(columns={"Nacionality": "Nationality"}, inplace = True)

In [8]:
# Let's previw the dataset
df.head(5)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


Now, I will view the column names to check for leading and trailing spaces.

In [9]:
# view the column names of the dataframe
col_names = df.columns

col_names

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance\t', 'Previous qualification',
       'Previous qualification (grade)', 'Nationality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd

We need to modify columns' names in order to deal more efficiently with them.

In [12]:
# Let's remove leading and trailing spaces
df.columns = df.columns.str.strip()

# Rename the columns
df.columns = [
    'marital_status','application_mode','application_order','course_id','daytime_attendance','prev_qualification','prev_qual_grade',
    'nationality','mother_qual','father_qual','mother_occupation','father_occupation','admission_grade',
    'is_displaced','special_needs','is_debtor','fees_paid','gender','has_scholarship','age_enrollment',
    'is_international','sem1_units_credited','sem1_units_enrolled','sem1_units_eval','sem1_units_approved',
    'sem1_grade_avg','sem1_units_no_eval','sem2_units_credited','sem2_units_enrolled','sem2_units_eval',
    'sem2_units_approved','sem2_grade_avg','sem2_units_no_eval','unemployment_rate','inflation_rate','gdp','target'
]

# View the renamed columns
df.columns

Index(['marital_status', 'application_mode', 'application_order', 'course_id',
       'daytime_attendance', 'prev_qualification', 'prev_qual_grade',
       'nationality', 'mother_qual', 'father_qual', 'mother_occupation',
       'father_occupation', 'admission_grade', 'is_displaced', 'special_needs',
       'is_debtor', 'fees_paid', 'gender', 'has_scholarship', 'age_enrollment',
       'is_international', 'sem1_units_credited', 'sem1_units_enrolled',
       'sem1_units_eval', 'sem1_units_approved', 'sem1_grade_avg',
       'sem1_units_no_eval', 'sem2_units_credited', 'sem2_units_enrolled',
       'sem2_units_eval', 'sem2_units_approved', 'sem2_grade_avg',
       'sem2_units_no_eval', 'unemployment_rate', 'inflation_rate', 'gdp',
       'target'],
      dtype='object')

In [25]:
# Let's check the distribution of our target variable
df["target"].value_counts()

,count
target,
Graduate,2209
Dropout,1421
Enrolled,794


In [26]:
# I want to represent them as percentage
perc_target =  round((df["target"].value_counts() / len(df)) * 100, ndigits = 2)
perc_target

,count
target,
Graduate,49.93
Dropout,32.12
Enrolled,17.95


In [28]:
# I want to have a sense of the absolute number of enrolled instances
print((df["target"] == "Enrolled").sum())

794


In order to reduce the classification problem to a binary problem, we may consider to drop rows where the target is equal to "Enrolled". We would lost only the ≈ 18% of the observation.

In [29]:
# Eliminate the rows where target = "Enrolled"
df.drop(labels = df[df["target"] == "Enrolled"].index, inplace = True)

df["target"].value_counts()

,count
target,
Graduate,2209
Dropout,1421


In [31]:
# Let's reset the index and get a sense of our data frame
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3630 entries, 0 to 3629
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   marital_status       3630 non-null   int64  
 1   application_mode     3630 non-null   int64  
 2   application_order    3630 non-null   int64  
 3   course_id            3630 non-null   int64  
 4   daytime_attendance   3630 non-null   int64  
 5   prev_qualification   3630 non-null   int64  
 6   prev_qual_grade      3630 non-null   float64
 7   nationality          3630 non-null   int64  
 8   mother_qual          3630 non-null   int64  
 9   father_qual          3630 non-null   int64  
 10  mother_occupation    3630 non-null   int64  
 11  father_occupation    3630 non-null   int64  
 12  admission_grade      3630 non-null   float64
 13  is_displaced         3630 non-null   int64  
 14  special_needs        3630 non-null   int64  
 15  is_debtor            3630 non-null   i